# Bottle Processing
Author: Andrew Reed

### Motivation:
Independent verification of the suite of physical and chemical observations provided by OOI are critical for the observations to be of use for scientifically valid investigations. Consequently, CTD casts and Niskin water samples are made during deployment and recovery of OOI platforms, vehicles, and instrumentation. The water samples are subsequently analyzed by independent labs for  comparison with the OOI telemetered and recovered data.

However, currently the water sample data routinely collected and analyzed as part of the OOI program are not available in a standardized format which maps the different chemical analyses to the physical measurements taken at bottle closure. Our aim is to make these physical and chemical analyses of collected water samples available to the end-user in a standardized format for easy comprehension and use, while maintaining the source data files. 

### Approach:
Generating a summary of the water sample analyses involves preprocessing and concatenating multiple data sources, and accurately matching samples with each other. To do this, I first preprocess the ctd casts to generate bottle (.btl) files using the SeaBird vendor software following the SOP available on Alfresco. 

Next, the bottle files are parsed using python code and the data renamed following SeaBird's naming guide. This creates a series of individual cast summary (.sum) files. These files are then loaded into pandas dataframes, appended to each other, and exported as a csv file containing all of the bottle data in a single data file.

### Data Sources/Software:

* **sbe_name_map**: This is a spreadsheet which maps the short names generated by the SeaBird SBE DataProcessing Software to the associated full names. The name mapping originates from SeaBird's SBE DataProcessing support documentation.

* **Alfresco**: The Alfresco CMS for OOI at alfresco.oceanobservatories.org is the source of the ctd hex, xmlcon, and psa files necessary for generating the bottle files needed to create the sample summary sheet.

* **SBEDataProcessing-Win32**: SeaBird vendor software for processing the raw ctd files and generating the .btl files.


**========================================================================================================================**
Import packages which will be used in this notebook:

In [1]:
import os, sys, re
import pandas as pd
import numpy as np

Load the name mapping for the column names based on SeaBird's manual:

In [4]:
sbe_name_map = pd.read_excel('/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')

In [5]:
sbe_name_map.head()

,Short Name,Full Name,Friendly Name,Units,Notes/Comments
0,accM,Acceleration [m/s^2],acc M,m/s^2,NaN
1,accF,Acceleration [ft/s^2],acc F,ft/s^2,NaN
2,altM,Altimeter [m],alt M,m,NaN
3,altF,Altimeter [ft],alt F,ft,NaN
4,avgsvCM,"Average Sound Velocity [Chen-Millero, m/s]",avgsv-C M,"Chen-Millero, m/s",NaN


Specify the directories where the different data sets are stored locally:

In [7]:
basepath = '/home/andrew/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/'
array = 'Irminger/'
cruise = 'Irminger-3/'
leg = ''
water_dir = 'Water Sampling/'
ctd_dir = 'ctd/'

In [8]:
#os.listdir(basepath+array+cruise+water_dir)
os.listdir(basepath+array+cruise+water_dir)

['Salts and O2',
 'CTD_Station_Log_Sta002.docx',
 'CTD_Station_Log_Sta003.docx',
 'Irminger_Sea-03_AR-07-01_DIC_Sample_Data_2019-01-02_ver_1-00.xlsx',
 'Irminger_Sea-03_AR-07-01_CTD_Sampling_Log_2017-09-28_ver_1-00.xlsx',
 'Irminger_Sea-03_AR-07-01_Nutrients_Sample_Data_2016-11-16_ver-1-00.xlsx',
 'CTD_Station_Log_Sta004.docx']

In [ ]:
files = os.listdir(basepath+array+cruise+water_dir)
for f in files:
    if 'Leg_2' in f:
        print(f)


Create the full directory paths for the relevant data:

In [ ]:
# Specify the local directory where the bottle (.btl) files are stored for a particular cruise
# dirpath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/ctd/'
btlpath = basepath+array+cruise+leg+ctd_dir
summary_sheet_path = basepath+array+cruise+water_dir+'Irminger_Sea-02_AT-30-01_CTD_Sampling_Logs_2016-07-12_ver_1-00.xlsx'
salts_and_o2_path = basepath+array+cruise+water_dir+'Salts and O2/'+leg
nutrients_path = basepath+array+cruise+water_dir+'Irminger_Sea-02_AT-30-01_Nutrients_Sample_Data_2016-09-01_ver_1-00.xlsx'
chl_path = basepath+array+cruise+water_dir+'Irminger_Sea-02_AT-30-01_Chlorophyll_Sample_Data_2017-09-21_ver_1-00.xlsx'
dic_path = basepath+array+cruise+water_dir+'Irminger_Sea-02_AT-30-01_DIC_Sample_Data_2019-01-02_ver_1-00.xlsx'

In [ ]:
# Parse the data for the start_time
def parse_header(header):
    """
    Parse the header of bottle (.btl) files to get critical information
    for the summary spreadsheet.
    
    Args:
        header - an object containing the header of the bottle file as a list of
            strings, split at the newline.
    Returns:
        hdr - a dictionary object containing the start_time, filename, latitude,
            longitude, and cruise id.
    """
    hdr = {}
    for line in header:
        if 'start_time' in line.lower():
            start_time = pd.to_datetime(re.split('= |\[',line)[1])
            hdr.update({'Start Time [UTC]':start_time.strftime('%Y-%m-%dT%H:%M:%SZ')})
        elif 'filename' in line.lower():
            hex_name = re.split('=',line)[1].strip()
            hdr.update({'Filename':hex_name})
        elif 'latitude' in line.lower():
            start_lat = re.split('=',line)[1].strip()
            hdr.update({'Start Latitude [degrees]':start_lat})
        elif 'longitude' in line.lower():
            start_lon = re.split('=',line)[1].strip()
            hdr.update({'Start Longitude [degrees]':start_lon})
        elif 'cruise id' in line.lower():
            cruise_id = re.split(':',line)[1].strip()
            hdr.update({'Cruise':cruise_id})
        else:
            pass
    
    return hdr
        

In [ ]:
# Now write a function to autopopulate the bottle summary sample sheet
files = [x for x in os.listdir(btlpath) if '.btl' in x]
for filename in files:
    filepath = os.path.abspath(btlpath+filename)
    
    # Load the raw content into memory
    with open(filepath) as file:
        content = file.readlines()
    content = [x.strip() for x in content]
    
    # Now parse the file content
    header = []
    columns = []
    data = []
    for line in content:
        if line.startswith('*') or line.startswith('#'):
            header.append(line)
        else:
            try:
                float(line[0])
                data.append(line)
            except:
                columns.append(line)
    
    # Parse the header
    hdr = parse_header(header)
    
    # Parse the column identifiers
    column_dict = {}
    for line in columns:
        for i,x in enumerate(line.split()):
            try:
                column_dict[i] = column_dict[i] + ' ' + x
            except:
                column_dict.update({i:x})
    
    # Parse the bottle data based on the column header locations
    data_dict = {x:[] for x in column_dict.keys()}

    for line in data:
        if line.endswith('(avg)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            for i,x in enumerate(values):
                data_dict[i].append(x)
        elif line.endswith('(sdev)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            data_dict[1].append(values[0])
        else:
            pass
            
    data_dict[1] = [' '.join(item) for item in zip(data_dict[1][::2],data_dict[1][1::2])]
    
    # With the parsed data and column names, match up the data and column
    # based on the location
    results = {}
    for key,item in column_dict.items():
        values = data_dict[key]
        results.update({item:values})
        
    # Put the results into a dataframe
    df = pd.DataFrame.from_dict(results)
        
    # Now add the parsed info from the header files into the dataframe
    for key,item in hdr.items():
        df[key] = item
        
    # Get the cast number
    cast = filename[filename.index('.')-3:filename.index('.')]
    df['Cast'] = str(cast).zfill(3)
    
    # Generate a filename for the summary file
    outname = filename.split('.')[0] + '.sum'
    
    # Save the results
    df.to_csv(btlpath+outname)

In [ ]:
# Now, for each "summary" file, load and append to each other
df = pd.DataFrame()
for file in os.listdir(btlpath):
    if '.sum' in file:
        df = df.append(pd.read_csv(btlpath+file))
    else:
        pass

In [ ]:
df.info()

In [ ]:
sbe_name_map['Short Name'].apply(lambda x: str(x).lower());

In [ ]:
# Rename the column title using the sbe_name_mapping 
for colname in list(df.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        df.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [ ]:
df

In [ ]:
df['Bottle Position'] = df['Bottle Position'].apply(lambda x: str( int(x) ) )
df.drop(columns='Unnamed: 0',inplace=True)
df['Cast'] = df['Cast'].apply(lambda x: str(x).zfill(3) )

In [ ]:
df

In [ ]:
df.to_csv(btlpath+'CTD_Summary.csv')

### Oxygen & Salinity 
Now, we need to add the 

In [ ]:
def clean_sal_files(dirpath):

    # Run check if files are held in excel format or csvs
    csv_flag = any(files.endswith('.SAL') for files in os.listdir(dirpath))
    if csv_flag:
        for filename in os.listdir(dirpath):
            sample = []
            salinity = []
            if filename.endswith('.SAL'):
                with open(dirpath+filename) as file:
                    data = file.readlines()
                    for ind1,line in enumerate(data):
                        if ind1 == 0:
                            strs = data[0].replace('"','').split(',')
                            cruisename = strs[0]
                            station = strs[1]
                            cast = strs[2]
                            case = strs[8]
                        elif int(line.split()[0]) == 0:
                            pass
                        else:
                            strs = line.split()
                            sample.append(strs[0])
                            salinity.append(strs[2])
                
                    # Generate a pandas dataframe to populate data
                    data_dict = {'Cruise':cruisename,'Station':station,'Cast':cast,'Case':case,'Sample ID':sample,'Salinity [psu]':salinity}
                    df = pd.DataFrame.from_dict(data_dict)
                    df.to_csv(file.name.replace('.','')+'.csv')
            else:
                pass
    
    else:
        # If the files are already in excel spreadsheets, they've been cleaned into a
        # logical tabular format
        pass
    

def process_sal_files(dirpath):
    
    # Check if the files are excel files or not
    excel_flag = any(files.endswith('SAL.xlsx') for files in os.listdir(dirpath))
    # Initialize a dataframe for processing the salinity files
    df = pd.DataFrame()
    if excel_flag:
        for file in os.listdir(dirpath):
            if 'SAL.xlsx' in file:
                df = df.append(pd.read_excel(dirpath+file))
        df.rename({'Sample':'Sample ID','Salinity':'Salinity [psu]','Niskin #':'Niskin','Case ID':'Case'}, 
                  axis='columns',inplace=True)
        df.dropna(inplace=True)
        df['Station'] = df['Station'].apply(lambda x: str( int(x)).zfill(3))
        df['Niskin'] = df['Niskin'].apply(lambda x: str( int(x)))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    else:
        for file in os.listdir(dirpath):
            if 'SAL.csv' in file:
                df = df.append(pd.read_csv(dirpath+file))
        df.dropna(inplace=True)
        df['Station'] = df['Station'].apply(lambda x: str( int(x)).zfill(3))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
        df.drop(columns=[x for x in list(df.columns.values) if 'unnamed' in x.lower()],inplace=True)

    # Save the processed summary file for salinity
    df.to_csv(dirpath+'SAL_Summary.csv')
    
    
def process_oxy_files(dirpath):
    df = pd.DataFrame()
    for filename in os.listdir(dirpath):
        if 'oxy' in filename.lower() and filename.endswith('.xlsx'):
            df = df.append(pd.read_excel(dirpath+filename)) 
            # Rename and clean up the oxygen data to be uniform across data sets
    df.rename({'Niskin #':'Niskin','Sample#':'Sample ID','Oxy':'Oxygen [mL/L]','Unit':'Units'},
              axis='columns',inplace=True)
    df.dropna(inplace=True)
    df['Station'] = df['Station'].apply(lambda x: str( int(x)).zfill(3))
    df['Niskin'] = df['Niskin'].apply(lambda x: str( int(x)))
    df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    df['Cruise'] = df['Cruise'].apply(lambda x: x.replace('O','0'))
    
    # Save the processed summary file for oxygen
    df.to_csv(dirpath+'OXY_Summary.csv')

In [ ]:
# Now process the salts and oxygen data
    # Clean the salinity
clean_sal_files(salts_and_o2_path)
    # Process the salinity files
process_sal_files(salts_and_o2_path)
    # Process the oxygen files
process_oxy_files(salts_and_o2_path)

### CTD Sampling Log
Load in the CTD sampling log summary sheet. The summary sheet needs to be manually created and the data cleaned before attempting to import. Additionally, ensure that there is only one header line and that it is at the top of the file.

In [ ]:
del sample_log

In [ ]:
sample_log = pd.read_excel(summary_sheet_path,sheet_name='Summary',header=0)
sample_log.head()

Rename the Comments field:

In [ ]:
sample_log.rename(columns={'Chlorophyll Comments':'Comments'},inplace=True)
sample_log.head()

In [ ]:
sample_log.info()

### Nutrient & Chlorophyll Data

In [ ]:
try:
    nutrients = pd.read_excel(nutrients_path,header=0)
    nutrients
except IsADirectoryError:
    nutrients = pd.DataFrame(data=sample_log['Nitrate Bottle 1'])
    nutrients.rename(columns={'Nitrate Bottle 1':'Sample ID'}, inplace=True)
    columns = ['Sample ID','Cruise','Avg: Nitrate + Nitrite [µmol/L]','Avg: Ammonium [µmol/L]',
               'Avg: Phosphate [µmol/L]','Avg: Silicate [µmol/L]','Avg: Nitrite [µmol/L]','Avg: Nitrate [µmol/L]']
    for col in columns:
        if col not in nutrients.columns.values:
            nutrients[col] = nutrients['Sample ID']

In [ ]:
nutrients.rename(columns={'Unnamed: 0':'Sample ID'}, inplace=True)

In [ ]:
nutrients.dropna(inplace=True)

In [ ]:
nutrients

In [ ]:
del chl

In [ ]:
sample_log['Chlorophyll Filter Sample # \nCast #/Depth/Bottle #/\n']

In [ ]:
try:
    chl = pd.read_excel(chl_path)
    chl.head()
except IsADirectoryError:
    # If there is no chlorophyll sheet yet, need to copy the bottle data into the final sample log
    chl = sample_log[['Station-Cast #','Chlorophyll Brown Bottle #','Chlorophyll Filter Sample #','Chlorophyll LN Tube']]
    chl.rename(columns={
        'Chlorophyll Brown Bottle #': 'Brown Bottle #',
        'Chlorophyll Filter Sample #': 'Chl (ug/l)',
        'Chlorophyll LN Tube':'Phaeo (ug/l)'
    }, inplace=True)

In [ ]:
chl.info()

In [ ]:
chl

In [ ]:
# Load the Salinity and oxygen summaries
sal = pd.read_csv(salts_and_o2_path+'SAL_Summary.csv')
if 'case' in [x.lower() for x in sal.columns.values]:
    sal['Sample ID'] = sal['Case'] + sal['Sample ID'].apply(lambda x: str(x)) 
oxy = pd.read_csv(salts_and_o2_path+'OXY_Summary.csv')
if 'case' in [x.lower() for x in oxy.columns.values]:
    oxy['Sample ID'] = oxy['Case'] + oxy['Sample ID'].apply(lambda x: str(x)) 

**========================================================================================================================**
### Carbon-System Measurements

In [ ]:
try:
    dic = pd.read_excel(dic_path,header=0)
    dic
except IsADirectoryError:
    dic = sample_log[['Station-Cast #','Niskin #','Ph Bottle #','DIC/TA Bottle #']]
    dic.rename(columns={
        'Station-Cast #':'CAST_NO',
        'Niskin #':'NISKIN_NO',
        'DIC/TA Bottle #':'DIC_UMOL_KG',
        'Ph Bottle #':'PH_TOT_MEA',
    }, inplace=True)
    columns = ['CAST_NO', 'NISKIN_NO','DIC_UMOL_KG', 'DIC_FLAG_W', 'TA_UMOL_KG',
       'TA_FLAG_W', 'PH_TOT_MEA', 'TMP_PH_DEG_C', 'PH_FLAG_W']
    for col in columns:
        if col not in dic.columns.values:
            if 'dic' in col.lower() or 'ta' in col.lower():
                dic[col] = dic['DIC_UMOL_KG']
            elif 'ph' in col.lower():
                dic[col] = dic['PH_TOT_MEA']
            else:
                dic[col] = np.nan

In [ ]:
dic.info()

**========================================================================================================================**
### Sample Log 
Next, we need to merge the sample log with the individual oxygen, salinity, nutrient, chlorophyll, and carbon sampling sheets.

In [ ]:
sample_log.columns.values

**========================================================================================================================**
Merge the **salinity** information with the sample_log based on cast # and salts sampling bottle:

In [ ]:
# Now need to mak
sample_log = sample_log.merge(sal[['Station','Sample ID','Salinity [psu]']], how='left', left_on=['Station-Cast #','Salts Bottle #'], right_on=['Station','Sample ID'])

In [ ]:
sample_log.rename({'Salinity [psu]':'Discrete Salinity [psu]'},axis='columns',inplace=True)
sample_log.drop(['Station','Sample ID'],axis='columns',inplace=True)

In [ ]:
sample_log.rename(columns=lambda x: x.strip(),inplace=True)

**========================================================================================================================**
Next, merge the **oxygen** data into the sample log based on cast # and oxygen sampling bottle:

In [ ]:
sample_log = sample_log.merge(oxy[['Station','Sample ID','Oxygen [mL/L]']], how='left', left_on=['Station-Cast #','Oxygen Bottle #'], right_on=['Station','Sample ID'])

In [ ]:
sample_log.rename({'Oxygen [mL/L]':'Discrete Oxygen [mL/L]'},axis='columns',inplace=True)
sample_log.drop(['Station','Sample ID'],axis='columns',inplace=True)

In [ ]:
sample_log.info()

**========================================================================================================================**
Merge the **nutrients** data into the sample log:

In [ ]:
nutrients.info()

In [ ]:
nutrients.reset_index(inplace=True)

In [ ]:
sample_log['Nitrate Bottle 1'] = sample_log['Nitrate Bottle 1'].apply(lambda x: x.strip() if type(x) == str else x)
sample_log.info()

In [ ]:
sample_log = sample_log.merge(nutrients, how='left', left_on=['Nitrate Bottle 1'], right_on=['Sample ID'])

In [ ]:
sample_log.drop_duplicates(inplace=True)

Rename the avg values to discrete, and drop unneeded columns:

In [ ]:
sample_log.rename(columns=lambda x: x.replace('Avg:', 'Discrete'), inplace=True)
sample_log.drop(['Sample ID'],axis='columns',inplace=True)

In [ ]:
sample_log.info()

**========================================================================================================================**
Merge the **chlorophyll** data into the sampling sheet:

In [ ]:
chl.columns.values

In [ ]:
chl_df = chl[['Station-Cast #','Brown Bottle #','Chl (ug/l)','Phaeo (ug/l)']]#,'Comments']]
chl_df.rename(columns={'Comments':'Chl Comments'}, inplace = True)
chl_df.rename(columns=lambda x: 'Discrete ' + x, inplace=True)
#chl_df.rename({'Discrete quality_flag':'Discrete Chl quality flag'},axis='columns',inplace=True)

In [ ]:
chl.info()

In [ ]:
sample_log = sample_log.merge(chl_df, how='left', left_on=['Station-Cast #','Chlorophyll Brown Bottle #'], right_on=['Discrete Station-Cast #','Discrete Brown Bottle #'])
sample_log.drop(['Discrete Station-Cast #','Discrete Brown Bottle #'],axis='columns',inplace=True)

In [ ]:
sample_log.info()

**========================================================================================================================**
Merge the **Carbon** data into the sampling sheet

In [ ]:
dic_df = dic[['CAST_NO', 'NISKIN_NO','DIC_UMOL_KG', 'DIC_FLAG_W', 'TA_UMOL_KG',
       'TA_FLAG_W', 'PH_TOT_MEA', 'TMP_PH_DEG_C', 'PH_FLAG_W']]
dic_df.rename(columns = {'DIC_UMOL_KG':'DIC [µmol/kg]',
               'DIC_FLAG_W':'DIC Flag',
               'TA_UMOL_KG':'Alkalinity [µmol/kg]',
               'TA_FLAG_W':'Alkalinity Flag',
               'PH_TOT_MEA':'pH [Total Scale]',
               'TMP_PH_DEG_C':'pH Analysis Temp [C]', 
              'PH_FLAG_W':'pH Flag'}, inplace=True)
# Add in the pCO2 columns, which we don't measure
dic_df['pCO2'] = np.nan
dic_df['pCO2 Flag'] = np.nan
dic_df['pCO2 Analysis Temp [C]'] = np.nan

dic_df.rename(columns=lambda x: 'Discrete ' + x, inplace=True)

In [ ]:
sample_log = sample_log.merge(dic_df, how='left', left_on=['Station-Cast #','Niskin #'], right_on=['Discrete CAST_NO','Discrete NISKIN_NO'])
sample_log.drop(['Discrete CAST_NO','Discrete NISKIN_NO'], axis='columns', inplace=True)

In [ ]:
sample_log.drop_duplicates(inplace=True)

In [ ]:
sample_log.info()

**========================================================================================================================**
### CTD Data
Now, we want to load the CTD bottle summary data and merge it with the water sampling data in the sample log.

In [ ]:
CTD = pd.read_csv(basepath+array+cruise+leg+ctd_dir+'CTD_Summary.csv')

In [ ]:
sample_log.rename({'Target Station':'Target Asset'},axis='columns',inplace=True)

In [ ]:
sample_log.info()

Create a list of columns to merge from the water sampling log with the CTD bottle summary data:

In [ ]:
column_list = []
for name in list(sample_log.columns.values):
    if 'Discrete' in name:
        column_list.append(name)
column_list.append('Station-Cast #')
column_list.append('Start Latitude')
column_list.append('Start Longitude')
column_list.append('Start Date')
column_list.append('Start Time')
column_list.append('Niskin #')
column_list.append('Target Asset')
column_list.append('Bottom Depth [m]')
column_list.append('Comments')

Use the column list to pull out the discrete data from the sample log:

In [ ]:
discrete_data = sample_log[column_list]

In [ ]:
discrete_data.info()

In [ ]:
discrete_data['Station-Cast #'] = discrete_data['Station-Cast #'].apply(lambda x: str(int(x)).zfill(3))

In [ ]:
CTD['Cast'] = CTD['Cast'].apply(lambda x: str(x).zfill(3) )

Merge the discrete data into the CTD data based on the Cast and Niskin bottle number:

In [ ]:
CTD = CTD.merge(discrete_data, how='left', left_on=['Cast','Bottle Position'], right_on=['Station-Cast #','Niskin #'])

In [ ]:
CTD.columns.values

In [ ]:
CTD.drop(labels=['Unnamed: 0','Niskin #'],axis='columns',inplace=True)

In [ ]:
CTD.rename({'Cast #':'Station'},inplace=True)

In [ ]:
CTD

In [ ]:
CTD.fillna(-9999999, inplace=True)
CTD['Cruise ID'] = sample_log['Cruise ID'][0]
CTD['Bottle Closure Time [UTC]'] = CTD['Date Time'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%dT%H:%M:%SZ'))
CTD.drop(columns='Date Time', inplace=True)

Now, find where there were data casts only (so no bottle closures), and fill in missing data on Start Latitutde, Longitude, Time, and Bottome Depth:

In [ ]:
CTD['Start Latitude [degrees]'] = CTD['Start Latitude'].where(CTD['Start Latitude [degrees]'] == -9999999, other=CTD['Start Latitude [degrees]'])
CTD['Start Longitude [degrees]'] = CTD['Start Longitude'].where(CTD['Start Longitude [degrees]'] == -9999999, other=CTD['Start Longitude [degrees]'])

In [ ]:
CTD['Station-Cast #'] = CTD['Cast'].where(CTD['Station-Cast #'] == -9999999, other=CTD['Station-Cast #'])

In [ ]:
CTD.drop_duplicates(inplace=True)

Autogenerate the filename following the agreed-upon form:

In [15]:
cruise_name = 'Irminger-3'
cruise_id = list(set(CTD['Cruise ID']))[0]
current_date = pd.to_datetime(pd.datetime.now()).tz_localize(tz='US/Eastern').tz_convert(tz='UTC')
version = '1-01'

NameError: name 'CTD' is not defined

In [ ]:
filename = '_'.join([cruise_name,cruise_id,'Discrete','Summary',current_date.strftime('%Y-%m-%d'),'ver',version,'.xlsx'])
filename

**========================================================================================================================**
Import the column order list and use fuzzy string matching to sort the data and save the data to an new Excel spreadsheet.

In [ ]:
column_order = pd.read_excel(basepath+'column_order.xlsx')

In [ ]:
column_order = tuple([x.replace('CTD','').strip() for x in column_order.columns.values])

In [ ]:
column_order

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
results = {}
CTDsorted = pd.DataFrame()
for column in column_order:
    match = process.extractBests(column.replace('Discrete ','').replace('Calculated ',''),
                                 CTD.columns.values, limit=2, score_cutoff=56, scorer=fuzz.ratio)
    if 'calculated' in column.lower():
        CTDsorted[column] = -9999999
    elif 'flag' in column.lower():
        if column not in ['Discrete DIC Flag','Discrete Alkalinity Flag','Discrete pCO2 Flag','Discrete pH Flag']:
            CTDsorted[column] = -9999999
        else:
            CTDsorted[column] = CTD[column]
            results.update({column:match[0]})
    elif len(match) == 0:
        CTDsorted[column] = -9999999
    elif (match[0][0] not in [x[0] for x in results.values()]):
        CTDsorted[match[0][0]] = CTD[match[0][0]]
        results.update({column:match[0]})
    elif len(match) == 1:
        CTDsorted[match[0][0]] = CTD[match[0][0]]
        results.update({column:match[0]})
    else:
        CTDsorted[match[1][0]] = CTD[match[1][0]]
        results.update({column:match[1]})
CTDsorted['Comments'] = CTD['Comments']

Now, check each of the resulting columns for if the values have actually been put in. If they haven't, we want to substitute in the sample bottle number, etc

In [ ]:
# Create a mask for the values that are just surface bottle closures
A = CTDsorted['Target Asset'] == -9999999
#B = CTDsorted['Station-Cast #'] == '008'
#mask = np.logical_or(A,B)

In [ ]:
CTDsorted[A].sort_values(by=['Station-Cast #','Bottle Position'])

In [ ]:
CTDsorted.columns.values

In [ ]:
np.unique(CTDsorted[A]['Station-Cast #'])

In [ ]:
CTDsorted[A][CTDsorted[A]['Station-Cast #'] == '007']['Pressure, Digiquartz [db]']

In [ ]:
# Now save the results
CTDsorted.to_excel(basepath+array+cruise+filename)

In [ ]:
CTDsorted[CTDsorted['Target Asset'] == -9999999]

In [ ]:
CTDsorted.columns

In [ ]:
CTDsorted[CTDsorted['Target Asset'] == -9999999][['Discrete Phaeo (ug/l)', 'Discrete Fo/Fa Ratio',
       'Discrete Fluorescence Flag', 'Discrete Fluorescence Duplicate Flag']].iloc[50:100]

In [ ]:
np.unique(CTDsorted[mask]['Station-Cast #'])

In [ ]:
CTDsorted[A]['Station-Cast #']

In [ ]:
np.unique(CTDsorted[CTDsorted['Target Asset'] != -9999999]['Station-Cast #'])

In [ ]:
np.unique(CTD['Station-Cast #'])

In [ ]:
len(CTDsorted[CTDsorted['Station-Cast #'] == '008'])

In [9]:
os.listdir(os.getcwd())

['2017_Key_File.docx',
 'Pioneer',
 'Cabled-8_RR1713-RR1717_Discrete_Summary_2017-07-27-README.docx',
 'rosette_file_processing.ipynb',
 'Untitled.ipynb',
 'Irminger',
 'column_order.xlsx',
 '.ipynb_checkpoints',
 'btl_file_processing.ipynb']

In [10]:
os.listdir('/'.join((os.getcwd(),'Irminger','Irminger-3')))

['.~lock.Irminger-3_AR07-01_Discrete_Summary_2019-06-12_ver_1-01_.xlsx#',
 'ctd',
 'Irminger-3_AR07-01_Discrete_Summary_2019-06-12_ver_1-01_.xlsx',
 'Water Sampling']

In [12]:
df = pd.read_excel('/'.join((os.getcwd(),'Irminger','Irminger-3','Irminger-3_AR07-01_Discrete_Summary_2019-06-12_ver_1-01_.xlsx')))

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 77 columns):
Cruise ID                                        503 non-null object
Station-Cast #                                   503 non-null int64
Target Asset                                     503 non-null object
Start Latitude [degrees]                         503 non-null object
Start Longitude [degrees]                        503 non-null object
Start Time [UTC]                                 503 non-null object
Cast                                             503 non-null int64
Cast Flag                                        503 non-null int64
Bottom Depth [m]                                 503 non-null int64
CTD Filename                                     503 non-null object
CTD File Flag                                    503 non-null int64
Bottle Position                                  503 non-null int64
Niskin Flag                                      503 non-null int64
CTD B

In [18]:
df.drop_duplicates(inplace=True)

In [19]:
filename = '/'.join((os.getcwd(),'Irminger','Irminger-3','Irminger-3_AR07-01_Discrete_Summary_2019-06-15_ver_1-01_.xlsx'))

In [20]:
df.to_excel(filename)